In [52]:
from utils import *
from richutils import *
warnings.filterwarnings("ignore")

df = pd.read_csv('dataset.csv', index_col='Month', parse_dates=True,date_parser=lambda x: pd.to_datetime(x, format='%y-%b'))
df


,Index,Price,Production,Withdrawals,GlobalPrice,ExchangeRate,TempMax,TempAvg,Precipitation,Inflation
Month,,,,,,,,,,
2014-09-01,1,51.51,537770.0,1418420,0.1470,44.08,28,27,456.59,3.9
2014-10-01,2,51.46,1623075.0,1355836,0.1648,44.80,29,27,436.73,3.7
2014-11-01,3,51.28,2277043.0,2156729,0.1588,44.95,29,28,265.32,3.0
2014-12-01,4,51.65,2223994.0,1663150,0.1499,44.69,28,27,182.00,1.9
2015-01-01,5,51.19,2259613.0,1714490,0.1506,44.60,28,27,82.83,1.5
...,...,...,...,...,...,...,...,...,...,...
2024-04-01,116,85.85,1476667.0,1042072,0.2069,56.95,33,29,22.90,3.8
2024-05-01,117,85.54,590428.0,1177536,0.1880,57.76,32,29,130.78,3.9
2024-06-01,118,86.73,470766.0,1183388,0.1916,58.70,30,28,330.53,3.7


In [53]:
price = df['Price']
production = df['Production']
withdrawals = df['Withdrawals']
globalPrice = df['GlobalPrice']
exchangeRate = df['ExchangeRate']
temperature = df['TempMax']
precipitation = df['Precipitation']
inflation = df['Inflation']

In [54]:
correlation_matrix = df[['Price', 'Production', 'Withdrawals', 'GlobalPrice', 
                         'ExchangeRate', 'TempMax', 'Precipitation', 'Inflation']].corr()

# Display correlation of each exog with 'Price'
print(correlation_matrix['Price'].sort_values(ascending=False))

Price            1.000000
ExchangeRate     0.831227
Inflation        0.699607
GlobalPrice      0.655497
TempMax          0.203669
Precipitation    0.077998
Production      -0.179624
Withdrawals     -0.580029
Name: Price, dtype: float64


In [55]:
y = price 
exog = pd.concat([production, withdrawals, globalPrice, exchangeRate, temperature, precipitation, inflation], axis=1)
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

# Assume `exog` is your DataFrame of exogenous variables
# (drop any columns like 'Price' if present)
X = exog.copy()

# Add constant for VIF computation
from statsmodels.tools.tools import add_constant
X = add_constant(X)

# Compute VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


         Feature          VIF
0          const  1289.048246
1     Production     2.994073
2    Withdrawals     1.770679
3    GlobalPrice     1.436425
4   ExchangeRate     2.534571
5        TempMax     1.196730
6  Precipitation     2.560422
7      Inflation     2.233639


In [56]:
corr = df[['Price', 'Production', 'Withdrawals', 'GlobalPrice',
           'ExchangeRate', 'TempMax', 'Precipitation', 'Inflation']].corr()

# Display correlation of each exog with 'Price'
print(corr['Price'].sort_values(ascending=False))

Price            1.000000
ExchangeRate     0.831227
Inflation        0.699607
GlobalPrice      0.655497
TempMax          0.203669
Precipitation    0.077998
Production      -0.179624
Withdrawals     -0.580029
Name: Price, dtype: float64


In [57]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

split = 24
y_train, y_test = train_test_split(y, split)
exog_train, exog_test = train_test_split(exog, split)

# Full model
model_full = SARIMAX(y_train, exog=exog_train, order=(1, 1, 1))
results_full = model_full.fit(disp=False)

# Forecast
forecast_full = results_full.predict(start=len(y_train), end=len(y_train)+len(y_test)-1, exog=exog_test)

# Evaluate
rmse_full = np.sqrt(mean_squared_error(y_test, forecast_full))
mae_full = mean_absolute_error(y_test, forecast_full)
mape_full = mean_absolute_percentage_error(y_test, forecast_full) * 100

# Information criteria
aic_full = results_full.aic
bic_full = results_full.bic

# Print results
print("Full model RMSE:", rmse_full)
print("Full model MAE:", mae_full)
print("Full model MAPE:", mape_full)
print("Full model AIC:", aic_full)
print("Full model BIC:", bic_full)



Full model RMSE: 6.251378513276449
Full model MAE: 4.583310033683165
Full model MAPE: 5.205193778366434
Full model AIC: 443.8139786248924
Full model BIC: 469.3527475408978


In [58]:
# Drop insignificant columns (replace with actual column names)
insig_cols = ['GlobalPrice', 'ExchangeRate']  # example
exog_train_sig = exog_train.drop(columns=insig_cols)
exog_test_sig = exog_test.drop(columns=insig_cols)

# Reduced model
model_sig = SARIMAX(y_train, exog=exog_train_sig, order=(1, 1, 1))
results_sig = model_sig.fit(disp=False)

# Forecast
forecast_sig = results_sig.predict(start=len(y_train), end=len(y_train)+len(y_test)-1, exog=exog_test_sig)

# Evaluate
rmse_sig = np.sqrt(mean_squared_error(y_test, forecast_sig))
mae_sig = mean_absolute_error(y_test, forecast_sig)
mape_sig = mean_absolute_percentage_error(y_test, forecast_sig) * 100

aic_sig = results_sig.aic
bic_sig = results_sig.bic

print("Reduced model RMSE:", rmse_sig)
print("Reduced model MAE:", mae_sig)
print("Reduced model MAPE:", mape_sig)
print("Reduced model AIC:", aic_sig)
print("Reduced model BIC:", bic_sig)


Reduced model RMSE: 17.718053584883297
Reduced model MAE: 17.320993897214795
Reduced model MAPE: 18.508915989245192
Reduced model AIC: 668.2295307950586
Reduced model BIC: 688.6605459278629
